In [1]:
%pylab inline

import os
import pandas as pd
from collections import OrderedDict
from astropy.io import fits
from copy import deepcopy as copy
from scipy.optimize import leastsq
from styles import slides

Populating the interactive namespace from numpy and matplotlib


In [2]:
def nearest(a, b):
    return np.argmin(np.abs(a-b))

In [3]:
rdir = os.path.expandvars("$HOME/Research/photometric-ew")
flist = sorted([os.path.join(root,file) for root, subs, files in os.walk(rdir+"/data/fits") for file in files if file.endswith(".fit.gz")])

line_ids = pd.read_csv("{}/data/fits_hdu2.csv".format(rdir), index_col="ID")
line_ids.sort_values(by="restWave", inplace=True)

fits_nms = [name.split("/")[-1] for name in flist]

#rows = OrderedDict(zip(line_nms,line_ids))
ew = OrderedDict([(kw,[]) for kw in line_ids.index])
er = OrderedDict([(kw,[]) for kw in line_ids.index])
z, mags = [], OrderedDict([(kw,[]) for kw in "u g r i z".split()])

In [4]:
for fn in flist:
    with fits.open(fn) as f:
        z.append(f[0].header["z"])
        mag = map(eval,f[0].header["MAG"].split())
        for j,kw in enumerate(mags.keys()):
            mags[kw] += [mag[j]]
        
        for kw in line_ids.index:
            ew[kw] += [f[2].data["ew"][nearest(f[2].data["restWave"],line_ids.restWave[kw])]]
            er[kw] += [f[2].data["ewErr"][nearest(f[2].data["restWave"],line_ids.restWave[kw])]]

In [5]:
table_dat = OrderedDict()
table_dat["FITS"] = fits_nms
table_dat.update(ew)
table_dat.update(mags)
table_dat["redshift"] = z

df_dat = pd.DataFrame(table_dat)
df_dat.replace(-9999.0, np.nan, inplace=True)
df_dat.to_csv("{}/data/photo-speclines.csv".format(rdir), index=False)

table_err = OrderedDict()
table_err["FITS"] = fits_nms
table_err.update(er)

df_err = pd.DataFrame(table_err)
df_err.replace(-9999.0, np.nan, inplace=True)
df_err.to_csv("{}/data/photo-speclines_err.csv".format(rdir), index=False)